# EMT MADRID - MobilityLabs API
The docs of the API can be found here:
https://apidocs.emtmadrid.es/

To start using the API, it is necessary to register an account with an email and a password that allow us to login and get an access token, we can do that in the following website:
https://mobilitylabs.emtmadrid.es/

We also have access to some examples and the source code of the api in the following link: https://gitlab.com/mobilitylabsmadrid

The following notebook gives a simple idea of how to use the main methods of the API, but there are more available that can be found in the docs.

In [1]:
#First we import the requested modules
import pandas as pd
import json

import requests 
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

#Function to ensure we get a good response for the request
def requests_retry_session(retries=3,backoff_factor=0.3,status_forcelist=(500, 502, 504),session=None):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

## Start session in the API
Once we have our account, we can login and get an accessToken with the following method:

In [4]:
#Our access credentials
email = 'p.parker230220@gmail.com'
password = 'CkBLZvmg5Jg&pej64$'
email = 'alejarabo@gmail.com'
password = 'YXKE8fDrX.pMZ58'

#We make the request for the login, in order to get the access token
response = requests_retry_session().get(
    'https://openapi.emtmadrid.es/v2/mobilitylabs/user/login/',
    headers={
        'email':email,
        'password':password
    },
    timeout=5
)

#We transform the response to json
json_response = response.json()
json_response

{'code': '00',
 'description': 'Register user: alejarabo with token: f821ffc7-d292-417d-93fa-65b2509451f7 ',
 'datetime': '2020-03-27T12:14:10.212375',
 'data': [{'updatedAt': '2020-02-24T08:40:49.7200000',
   'username': 'alejarabo',
   'accessToken': 'f821ffc7-d292-417d-93fa-65b2509451f7',
   'tokenSecExpiration': 86399,
   'email': 'alejarabo@gmail.com',
   'idUser': '9e44cabc-d4bd-4cc7-bbc4-89ae4f8adaf5',
   'apiCounter': {'current': 0,
    'dailyUse': 20000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!',
    'aboutUses': 'If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylabs.emtmadrid.es/doc/new-app and https://apidocs.emtmadrid.es/#api-Block_1_User_identity-login)'},
   'nameApp': 'OPENAPI MobilityLabs',
   'priv': 'U'}]}

In [7]:
#We get the access token from the json and print it
accessToken = json_response['data'][0]['accessToken']
accessToken

'f821ffc7-d292-417d-93fa-65b2509451f7'

## Get the stops that belong to a line and a direction
Calling the following method we get the stops that a bus passes through when it goes through the specified line and direction

In [11]:
lineId = '1' #We want to get the stops for the line with id=1
direction = '1' #In the direction 1
response = requests_retry_session().get(
    'https://openapi.emtmadrid.es/v2/transport/busemtmad/lines/{}/stops/{}/'.format(lineId,direction),
    headers = {'accessToken': accessToken}
)

#We turn the response into a json and show the result of the request
line_stops = response.json()['data'][0]
line_stops

{'line': '001',
 'label': '1',
 'stops': [{'stop': '4514',
   'name': 'Cristo Rey',
   'postalAddress': 'Isaac Peral, frente al Nº 40.',
   'geometry': {'type': 'Point',
    'coordinates': [-3.71793052606063, 40.4384746558032]},
   'pmv': '61748',
   'dataLine': ['001', '044', '069', '082', '132']},
  {'stop': '4022',
   'name': 'Junta Municipal Moncloa',
   'postalAddress': 'Isaac Peral frente al Nº 8',
   'geometry': {'type': 'Point',
    'coordinates': [-3.71842134613126, 40.4358409821157]},
   'pmv': '61745',
   'dataLine': ['001', '044', '062', '069', '082', '132', '138', '521']},
  {'stop': '3687',
   'name': 'Moncloa',
   'postalAddress': 'Arcipreste de Hita con C/ Fernando el Catolico',
   'geometry': {'type': 'Point',
    'coordinates': [-3.7180849386363, 40.4341403665582]},
   'pmv': '61225',
   'dataLine': ['001', '044', '062', '069', '138', '521']},
  {'stop': '737',
   'name': 'Altamirano',
   'postalAddress': 'Princesa, 75',
   'geometry': {'type': 'Point',
    'coordinat

## Information about buses that are going to arrive a certain stop
With this method we can get a lot information about the buses that are going to arrive a stop, such as the ETA, the distance, the bus id and much more. We have to send a json in the data of the request to specify if we want to get or not information about the stop, the incidents and the estimations. If we dont specify the line we want to get the information about, it returns us the information about all the lines that pass through this stop.

In [16]:
#We get the id for the first stop of the line 1
stopId = line_stops['stops'][0]['stop']

#We construct the body for the request
body = {
    'cultureInfo': 'EN',
    'Text_StopRequired_YN': 'Y',
    'Text_EstimationsRequired_YN': 'Y',
    'Text_IncidencesRequired_YN': 'Y',
    'DateTime_Referenced_Incidencies_YYYYMMDD':'20200130'
}

#And we turn it into a json object
body = json.dumps(body)
body

'{"cultureInfo": "EN", "Text_StopRequired_YN": "Y", "Text_EstimationsRequired_YN": "Y", "Text_IncidencesRequired_YN": "Y", "DateTime_Referenced_Incidencies_YYYYMMDD": "20200130"}'

In [17]:
import re
import datetime
#And once we have all the attributes for the request ready, we perform it like this:
response = requests_retry_session().post(
    'https://openapi.emtmadrid.es/v2/transport/busemtmad/stops/{}/arrives/{}/'.format('162','1'),
    data = body,
    headers = {
        'accessToken': accessToken,
        'Content-Type': 'application/json' #We specify that we are doing an application with a json object
    },
    timeout = 5
)

#And we show the data returned by the request
response.status_code
arrival_data = response.json()
arrival_data['data'][0]

{'Arrive': [{'line': '1',
   'stop': '162',
   'isHead': 'False',
   'destination': 'PROSPERIDAD',
   'deviation': 0,
   'bus': 124,
   'geometry': {'type': 'Point',
    'coordinates': [-3.71810556101814, 40.433748896629005]},
   'estimateArrive': 624,
   'DistanceBus': 2740,
   'positionTypeBus': '0'},
  {'line': '1',
   'stop': '162',
   'isHead': 'False',
   'destination': 'PROSPERIDAD',
   'deviation': 0,
   'bus': 122,
   'geometry': {'type': 'Point',
    'coordinates': [-3.718160557144212, 40.4331440613206]},
   'estimateArrive': 1738,
   'DistanceBus': 4720,
   'positionTypeBus': '0'}],
 'StopInfo': [{'lines': [{'label': '1',
     'line': '001',
     'nameA': 'CRISTO REY',
     'nameB': 'PROSPERIDAD',
     'metersFromHeader': 3979,
     'to': 'B',
     'color': '0072ce'}],
   'stopId': '162',
   'stopName': 'Retiro',
   'geometry': {'type': 'Point',
    'coordinates': [-3.68826031065316, 40.4196998317684]},
   'Direction': 'Pza. de la Independencia (Retiro)                      

## Obtain collections of data
We can also get collections of data from the API, the collections available are shown in the following link:
https://mobilitylabs.emtmadrid.es/es/portal/collections 

In order to get the data of the desired collection, we have to get its collection id from the site above, and then call the following method

In [19]:
#We request the collection of Madrid Central Area, that has this id
collection_id = 'b94ea072-77ca-433d-adce-b3504f3abd00'
weekly_GTFS_id = '85aa7e85-d6b8-4cf9-a955-67a2b8056d67'
waiting_times = 'b087e295-8750-4f77-9672-30e9875af145'

#And we perform the request
response = requests_retry_session().post(
    'https://openapi.emtmadrid.es/v1/mobilitylabs/collection/reactive/{}/1/'.format(waiting_times),
    headers = {
        'accessToken': accessToken,
        'Content-Type': 'application/json'
    },
    data = {},
    timeout = 5
)

#We show the result of the request
response.text

'{"code": "90", "description": "Error in API process Expecting value: line 1 column 1 (char 0)", "datetime": "2020-03-21T17:32:22.622734", "data": []}'

## Create dictionary with stops for each line and direction
In order to make some methods faster, we build a dictionary that contains the stop codes and locations in an array for each line and direction.

In [9]:
def get_stops_of_line(lineId,direction,accessToken) :
    """
    Returns the list of stops for the line and direction desired

        Parameters
        ----------
        lineId : string
            The line id
        direction : string
            The direction (1 or 2)
        accessToken: string
            The accessToken obtained in the login
    """
    
    response = requests_retry_session().get(
        'https://openapi.emtmadrid.es/v2/transport/busemtmad/lines/{}/stops/{}/'.format(lineId,direction),
        headers = {'accessToken': accessToken},
        timeout = 5
    )
    
    #We turn the data of the stops from the response into a dataframe
    stops_data = pd.DataFrame(response.json()['data'][0]['stops'])   
    return stops_data

def get_destinations_of_line(lineId,accessToken) :
    """
    Returns the line destinations

        Parameters
        ----------
        lineId : string
            The line id
        accessToken: string
            The accessToken obtained in the login
    """
    
    response = requests_retry_session().get(
        'https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/{}/route/'.format(lineId),
        headers = {'accessToken': accessToken},
        timeout = 5
    )
    
    data = response.json()['data']
    #We get the destinations values
    nameA = data['nameSectionA']
    nameB = data['nameSectionB']
    
    return nameA,nameB

get_destinations_of_line(44,accessToken)

('PLAZA DEL CALLAO', 'MARQUES DE VIANA')

In [45]:
with open('M6Data/line_stops_dict.json', 'r') as f:
    line_stops_dict = json.load(f)

lines_shapes = pd.read_csv('M6Data/lines_shapes.csv')
#If we want to build the dictionary for all the stops we uncomment this (takes about 45min):
#line_ids = lines_shapes['line_id'].unique() 
#In order to make the example faster we just get the data for two lines
line_ids = ['1','2']
for lineId in line_ids:   
    if lineId not in line_stops_dict :
        line_stops_dict[lineId] = {}
        
    for direction in ['1','2']: 
        #For each line and direction we get the stops data and put it inside the dict
        try:
            if direction not in line_stops_dict[lineId] :
                line_stops_dict[lineId][direction] = {}
                stops_data = get_stops_of_line(lineId,direction,accessToken)
                line_stops_dict[lineId][direction] = stops_data['stop'].tolist()
        except :
            continue
#If we want to write the dict to a file            
#with open('M6Data/line_stops_dict.json', 'w') as fp:
    #json.dump(line_stops_dict, fp)